In [1]:
import pandas as pd
import numpy as np

import jieba
import jieba.posseg as pseg
import jieba.analyse

import sys
from optparse import OptionParser
import pprint
from prettytable import PrettyTable

In [6]:
df = pd.read_csv(open('../data/df_small.csv'))
df = df[df['text'].str.contains("https|Britain")== False]
print('有{}筆對話'.format(len(df)))

有91340筆對話


## groupby

In [4]:
grouped_data = df.groupby('from')['text'].apply(lambda x: '/'.join(x)).reset_index()
l = len(grouped_data)
print('有{}個使用者'.format(l))

有1019個使用者


## 先用前1000筆資料做測試

In [8]:
test = grouped_data.head(1000)
test.head()

,from,text
0,1000970273334199,找不到阿/安安/你有打LOL嗎/好吧，可以帶我嗎/安安/凸/Sor/嗯/約哪/有啊/你先約的...
1,1001070263323689,!?/唐寶寶/你那麼愛慕他ㄇ/嗨/先問問您期望本人的性別是?/了解/我是男的/抱歉沒有/請問...
2,1001967239901904,早/在床上發懶/爆/斷/魂/你在暗諷綠藻嘛/壞/話說我好餓ㄛ/窩不要這個qq/ㄋ早餐吃什麼/...
3,1003172849781848,甲/你要做我的0？/人生勝利組是不會來玩這個的/你一定不是c銘/也不是業王/葉你妹/被盜/其...
4,1003929993039131,吃噓/肥宅早/我們不是場外人/場外ㄈㄓ/我沒用了/安安/是喔/場外ㄈㄓ嗎/豪/斷/欸/在打手...


In [9]:
for i in range(len(test)):
    test.iloc[i][1] = test.iloc[i][1].split('/')
test.head()

,from,text
0,1000970273334199,"[找不到阿, 安安, 你有打LOL嗎, 好吧，可以帶我嗎, 安安, 凸, Sor, 嗯, 約..."
1,1001070263323689,"[!?, 唐寶寶, 你那麼愛慕他ㄇ, 嗨, 先問問您期望本人的性別是?, 了解, 我是男的,..."
2,1001967239901904,"[早, 在床上發懶, 爆, 斷, 魂, 你在暗諷綠藻嘛, 壞, 話說我好餓ㄛ, 窩不要這個q..."
3,1003172849781848,"[甲, 你要做我的0？, 人生勝利組是不會來玩這個的, 你一定不是c銘, 也不是業王, 葉你..."
4,1003929993039131,"[吃噓, 肥宅早, 我們不是場外人, 場外ㄈㄓ, 我沒用了, 安安, 是喔, 場外ㄈㄓ嗎, ..."


In [11]:
print('id為 {} 的使用者傳送的訊息{}'.format(test.iloc[0][0],test.iloc[0][1]))

id為 1000970273334199 的使用者傳送的訊息['找不到阿', '安安', '你有打LOL嗎', '好吧，可以帶我嗎', '安安', '凸', 'Sor', '嗯', '約哪', '有啊', '你先約的啊', '喔', '輸入0088', '我懶的打了', '皮米屌', '這你嗎？', '魏癢癢', '哪裡人?', '住哪裡啦？', '超癢', '哪裡人?', '好遠', '其實還好', '新竹', '龍潭?', 'ㄏㄅㄨㄐㄅㄌㄗㄋ', '三小', '幹又ㄋ', '掰掰', '凹?', '凸sor']


In [12]:
jieba.set_dictionary('jieba_txt/dict.txt.big.txt')
stopwordset = set()
with open('jieba_txt/stop_words.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
stopwordset

{'10',
 '20',
 '2016',
 '?',
 '他',
 '他們',
 '你',
 '台灣',
 '她',
 '對於',
 '成為',
 '我',
 '我們',
 '沒有',
 '王隆甫',
 '？'}

In [17]:
chars = set(' abcdefghijklmnopqrstuvwxyz0123456789()?$,:.。，-_&=/􀆿\r\n')
for c in chars:
    stopwordset.add(c)

In [19]:
# stopwordset

## gensim model

In [23]:
from gensim import models
import gensim

In [26]:
sentences = models.word2vec.LineSentence("corpusSegDone.txt")
model = models.word2vec.Word2Vec(sentences, size=1, window=5, min_count=5, workers=1)
model.wv.vocab

{'一個': <gensim.models.keyedvectors.Vocab at 0x7f5d4076d048>,
 '阿': <gensim.models.keyedvectors.Vocab at 0x7f5d4080e5c0>,
 '洞': <gensim.models.keyedvectors.Vocab at 0x7f5d409afcf8>,
 '關門': <gensim.models.keyedvectors.Vocab at 0x7f5d4094cbe0>,
 '真鹿': <gensim.models.keyedvectors.Vocab at 0x7f5d4080e828>,
 'all': <gensim.models.keyedvectors.Vocab at 0x7f5d4094cfd0>,
 '¯': <gensim.models.keyedvectors.Vocab at 0x7f5d40948668>,
 'ん': <gensim.models.keyedvectors.Vocab at 0x7f5d4097e4a8>,
 '試過': <gensim.models.keyedvectors.Vocab at 0x7f5d40a103c8>,
 '脫': <gensim.models.keyedvectors.Vocab at 0x7f5d40912630>,
 '安': <gensim.models.keyedvectors.Vocab at 0x7f5d409125f8>,
 'not': <gensim.models.keyedvectors.Vocab at 0x7f5d40a24a20>,
 '沒交過': <gensim.models.keyedvectors.Vocab at 0x7f5d40a2e400>,
 '你講': <gensim.models.keyedvectors.Vocab at 0x7f5d40771048>,
 '第三': <gensim.models.keyedvectors.Vocab at 0x7f5d409125c0>,
 '金': <gensim.models.keyedvectors.Vocab at 0x7f5d40a10400>,
 'ㄔ': <gensim.models.keyedve

## pick user top 10 word ( frequency-based )

In [93]:
top10_vector = np.zeros(shape=(20,10))
top10_vector

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [94]:
import operator
for i in range(20):
    og_dict = dict()
    print('\n user'+str(i+1))
    if (len(test.iloc[i][1]))<10:
        continue
    for j in range(len(test.iloc[i][1])):
        seglist = jieba.cut((test.iloc[i][1])[j])
        for word in seglist:
            if word not in stopwordset:
                if not word in og_dict.keys():
                    if word == " ":
                        continue
                    og_dict[word] = 1
                else:
                    og_dict[word] = int(og_dict[word]) + 1
    a = sorted(og_dict.items(), key=operator.itemgetter(1),reverse=True)
#     print(a)
    if len(a) >= 10:
        row = np.zeros(shape=(1,10))
        for x in range(10):
#             print(a[x]) 
            if a[x][0] in model:
                show = float(model[a[x][0]])
                print(show)
                row[0][x] = show
        print(row)
        top10_vector[i] = row


 user1
2.617060422897339
2.7319674491882324
0.07984180003404617
2.698899507522583
2.592031955718994
2.6054847240448
2.689040422439575
2.7317230701446533
2.5781054496765137
2.0049777030944824
[[ 2.61706042  2.73196745  0.0798418   2.69889951  2.59203196  2.60548472
   2.68904042  2.73172307  2.57810545  2.0049777 ]]

 user2
2.7317230701446533
2.617060422897339
2.6571874618530273
2.6054847240448
2.6508805751800537
2.4325554370880127
2.6125872135162354
2.583458662033081
2.518571615219116
[[ 2.73172307  2.61706042  2.65718746  2.60548472  2.65088058  2.43255544
   2.61258721  0.          2.58345866  2.51857162]]

 user3
2.6571874618530273
2.805999517440796
2.6386783123016357
2.6736481189727783
2.7317230701446533
2.5781054496765137
2.617060422897339
2.689040422439575
2.583458662033081
2.6869583129882812
[[ 2.65718746  2.80599952  2.63867831  2.67364812  2.73172307  2.57810545
   2.61706042  2.68904042  2.58345866  2.68695831]]

 user4
2.7317230701446533
2.658313751220703
2.7267963886260986

In [95]:
top10_vector.shape

(20, 10)

In [96]:
top10_vector

array([[ 2.61706042,  2.73196745,  0.0798418 ,  2.69889951,  2.59203196,
         2.60548472,  2.68904042,  2.73172307,  2.57810545,  2.0049777 ],
       [ 2.73172307,  2.61706042,  2.65718746,  2.60548472,  2.65088058,
         2.43255544,  2.61258721,  0.        ,  2.58345866,  2.51857162],
       [ 2.65718746,  2.80599952,  2.63867831,  2.67364812,  2.73172307,
         2.57810545,  2.61706042,  2.68904042,  2.58345866,  2.68695831],
       [ 2.73172307,  2.65831375,  2.72679639,  2.37619686,  1.17757142,
         1.12671244,  0.        ,  2.58085132,  0.        ,  0.86538321],
       [ 2.73172307,  2.67375588,  2.43255544,  2.36152649,  2.79830718,
         2.61706042,  2.56986117,  2.0049777 ,  2.10338235,  2.58345461],
       [ 2.61831498,  2.73172307,  2.58345866,  2.65718746,  2.64611149,
         2.45371199,  2.65516949,  2.69892836,  2.84651351,  2.59203196],
       [ 2.58345866,  2.73172307,  2.65718746,  2.61706042,  3.18807054,
         2.61258721,  2.61831498,  2.70759726

In [97]:
from sklearn import cluster, datasets

# KMeans 演算法
kmeans_fit = cluster.KMeans(n_clusters = 3).fit(top10_vector)

# 印出分群結果
cluster_labels = kmeans_fit.labels_
print("分群結果：")
print(cluster_labels)
print("---")



分群結果：
[1 1 1 2 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 1]
---
